## Write predictions

In [ ]:
# returns a vector that maps a user to the list of items to predict
function user_to_items(users::Vector, items::Vector, medium::String)
    user_to_count = zeros(Int32, num_users(medium), Threads.nthreads())
    @tprogress Threads.@threads for u in users
        user_to_count[u, Threads.threadid()] += 1
    end
    user_to_count = convert.(Int32, vec(sum(user_to_count, dims = 2)))

    utoa = Vector{Vector{Int32}}()
    @showprogress for u = 1:num_users(medium)
        push!(utoa, Vector{Int32}(undef, user_to_count[u]))
    end

    @showprogress for i = 1:length(users)
        u = users[i]
        a = items[i]
        utoa[u][user_to_count[u]] = a
        user_to_count[u] -= 1
    end
    utoa
end;

In [ ]:
function evaluate(hyp::Hyperparams, m, users::Vector{Int32}, items::Vector{Int32})
    # get model inputs
    global G = hyp
    CUDA.math_mode!(CUDA.FAST_MATH; precision = :TensorFloat32)
    m = m |> device
    utoa = user_to_items(users, items, hyp.medium)
    epoch = (
        get_epoch_inputs(
            G.input_data,
            G.task,
            G.medium,
            G.implicit,
            G.num_users,
            G.input_alphas,
        ),
        nothing,
        nothing,
        nothing,
    )
    activation = G.implicit ? softmax : identity
    out_users = Vector{Int32}(undef, length(users))
    out_items = Vector{Int32}(undef, length(users))
    out_ratings = fill(NaN32, length(out_users))
    out_idx = 1

    # compute predictions
    N = num_users(G.medium)
    @showprogress for iter = 1:Int(ceil(N / G.batch_size))
        batch, sampled_users = get_batch(epoch, iter, G.batch_size, 1:N, false)
        alpha = activation(m(batch[1])) |> cpu
        for j = 1:length(sampled_users)
            u = sampled_users[j]
            if length(utoa[u]) > 0
                item_mask = utoa[u]
                next_idx = out_idx + length(item_mask)
                out_users[out_idx:next_idx-1] .= u
                out_items[out_idx:next_idx-1] = item_mask
                out_ratings[out_idx:next_idx-1] = alpha[item_mask, j]
                out_idx = next_idx
            end
        end
    end

    global G = nothing
    CUDA.math_mode!(CUDA.FAST_MATH; precision = :BFloat16)
    RatingsDataset(
        user = out_users,
        item = out_items,
        rating = out_ratings,
        medium = hyp.medium,
    )
end;

In [ ]:
function write_alpha(hyp::Hyperparams, m, outdir::String)
    hyp = @set hyp.num_users = num_users(hyp.medium)
    test_users = union(
        [
            Set(get_split("test", hyp.task, x, hyp.medium; fields = [:user]).user) for
            x in ALL_CONTENTS
        ]...,
    )
    master_dfs = []
    for split in ALL_SPLITS
        @showprogress for content in ALL_CONTENTS
            df =
                get_raw_split(split, hyp.task, content, hyp.medium; fields = [:user, :item])
            if split == "training"
                df = filter(df, df.user .∈ (test_users,))
            end
            push!(master_dfs, df)
        end
    end
    master_df = reduce(cat, master_dfs)
    p = sparse(evaluate(hyp, m, master_df.user, master_df.item))
    function model(users, items)
        r = zeros(length(users))
        @tprogress Threads.@threads for j = 1:length(r)
            r[j] = p[items[j], users[j]]
        end
        r
    end
    write_alpha(
        model,
        hyp.medium,
        outdir;
        task = hyp.task,
        log = true,
        log_task = hyp.task,
        log_content = hyp.content,
        log_alphas = hyp.residual_alphas,
    )
end;